In [ ]:
!pip install kubernetes

In [1]:
from kubernetes import client, config
import json

In [ ]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_pod_for_all_namespaces(watch=False)
for i in ret.items:
    #print(i.status.conditions)
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

In [7]:
from time import sleep
as_v2 = client.AutoscalingV2Api()
metrics = [] # target metrics
current_metrics = [] # current metrics
replicas = [] # desired replicas and current replicas
while True:
  res = as_v2.list_namespaced_horizontal_pod_autoscaler('default')
  for item in res.items:
    for m in item.spec.metrics:
      # https://github.com/kubernetes-client/python/blob/master/kubernetes/docs/V2MetricSpec.md
      res = m.resource
      metrics.append(dict(
        target = dict(name = res.name, average_utilization=res.target.average_utilization, type=res.target.type),
      ))
    replicas.append(dict(
        current_replicas = item.status.current_replicas, 
        desired_replicas=item.status.desired_replicas, 
        last_scale_time=item.status.last_scale_time  
      )
    )
    for m in item.status.current_metrics:
      # https://github.com/kubernetes-client/python/blob/master/kubernetes/docs/V1HorizontalPodAutoscalerStatus.md
      res = m.resource
      current_metrics.append(dict(
        target = dict(name = res.name, average_utilization=res.current.average_utilization, average_value=res.current.average_value),
      ))
  sleep(1)
  break
  

In [57]:
res.items[0]

{'api_version': None,
 'kind': None,
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2022, 5, 6, 16, 51, 16, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': [{'api_version': 'autoscaling/v2',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'f:maxReplicas': {},
                                                           'f:metrics': {},
                                                           'f:minReplicas': {},
                                                           'f:scaleTargetRef': {'f:apiVersion': {},
                                                                                'f:kind': {},
       

In [30]:
# Get image size and pack into dict
image_sizes = {}
for node in v1.list_node().items:
  for image in node.status.images:
    for n in image.names:
      image_sizes[n] = image.size_bytes
image_sizes

{'k8s.gcr.io/kube-proxy:v1.23.1': 113372404,
 'k8s.gcr.io/etcd:3.5.1-0': 98888614,
 'k8s.gcr.io/kube-apiserver:v1.23.1': 76710844,
 'k8s.gcr.io/kube-controller-manager:v1.23.1': 65242402,
 'k8s.gcr.io/kube-scheduler:v1.23.1': 51917602,
 'docker.io/kindest/kindnetd:v20211122-a2c10462': 40928505,
 'k8s.gcr.io/build-image/debian-base:buster-v1.7.2': 21133992,
 'k8s.gcr.io/coredns/coredns:v1.8.6': 13585107,
 'docker.io/rancher/local-path-provisioner:v0.0.14': 13367922,
 'k8s.gcr.io/pause:3.6': 301773,
 'k8s.gcr.io/hpa-example@sha256:581697a37f0e136db86d6b30392f0db40ce99c8248a7044c770012f4e8491544': 164030864,
 'k8s.gcr.io/hpa-example:latest': 164030864,
 'k8s.gcr.io/metrics-server/metrics-server@sha256:5ddc6458eb95f5c70bd13fdab90cbd7d6ad1066e5b528ad1dcb28b76c5fb2f00': 28058350,
 'k8s.gcr.io/metrics-server/metrics-server:v0.6.1': 28058350}

In [31]:
def format_datetime(dt):
  return dt.strftime('%Y-%m-%dT%H:%M:%S%Z')

ret = v1.list_namespaced_pod("default")
pods = []
for i in ret.items:
  conditions = []
  for cond in i.status.conditions:
    conditions.append({
      'last_probe_time': cond.last_probe_time,
      'type': cond.type,
      'status': bool(cond.status),
      'last_transition_time': format_datetime(cond.last_transition_time)
    })
  image_name = i.status.container_statuses[-1].image
  pods.append({
    'name': i.metadata.name,
    'namespace': i.metadata.namespace,
    'container_statuses': {
      'image': i.status.container_statuses[-1].image,
      'image_size': image_sizes[image_name],
      'id': i.status.container_statuses[-1].image_id,
      'name': i.status.container_statuses[-1].name,
      'ready': i.status.container_statuses[-1].ready,
      'started': i.status.container_statuses[-1].started,
      'restart_count': i.status.container_statuses[-1].restart_count,
      'running_at': format_datetime(i.status.container_statuses[-1].state.running.started_at),
    },
    'phase': i.status.phase,
    'conditions': conditions,
    'start_time': format_datetime(i.status.start_time),
  })
  print(json.dumps(pods[-1]))


{"name": "php-apache-7656945b6b-77bnm", "namespace": "default", "container_statuses": {"image": "k8s.gcr.io/hpa-example:latest", "image_size": 164030864, "id": "k8s.gcr.io/hpa-example@sha256:581697a37f0e136db86d6b30392f0db40ce99c8248a7044c770012f4e8491544", "name": "php-apache", "ready": true, "started": true, "restart_count": 0, "running_at": "2022-05-06T16:51:06UTC"}, "phase": "Running", "conditions": [{"last_probe_time": null, "type": "Initialized", "status": true, "last_transition_time": "2022-05-06T16:50:51UTC"}, {"last_probe_time": null, "type": "Ready", "status": true, "last_transition_time": "2022-05-06T16:51:06UTC"}, {"last_probe_time": null, "type": "ContainersReady", "status": true, "last_transition_time": "2022-05-06T16:51:06UTC"}, {"last_probe_time": null, "type": "PodScheduled", "status": true, "last_transition_time": "2022-05-06T16:50:50UTC"}], "start_time": "2022-05-06T16:50:51UTC"}
{"name": "php-apache-7656945b6b-rxswf", "namespace": "default", "container_statuses": {"

In [3]:
ret = v1.list_pod_for_all_namespaces(watch=False)
ret.items[0]

{'api_version': None,
 'kind': None,
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2022, 5, 6, 16, 50, 50, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': 'php-apache-7656945b6b-',
              'generation': None,
              'labels': {'pod-template-hash': '7656945b6b',
                         'run': 'php-apache'},
              'managed_fields': [{'api_version': 'v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:metadata': {'f:generateName': {},
                                                               'f:labels': {'.': {},
                                                                            'f:pod-template-hash': {},
                                                                            'f: